# MODELO DE ISING - Algoritmo de Metrópolis

# ------------------------------------------------------

## Tarea:

Hacer la simulación y obtener los resultados con las gráficas de las diapositivas.

Obtener los exponentes del sistema sin el sesgo de confirmación de ya saber cuánto valen.

## Entrega:

Hacer un trabajo en latex sobre la física estudiada. No tiene interés en ver ningún código.


# ------------------------------------------------------

## Preámbulos

### Consideraciones:
- Debemos dar unos parámetros adecuados para cada temperatura. Lo que funcione bien para $T$ altas y bajas puede no funcionar bien cerca de la crítica
- A $T$ alta una buena configuración inicial puede ser aleatoria.
- A $T$ cercana a $T_C$ no hay una condicion inicial buena y sencilla. Podemos coger la configuración final de la temperatura más cercana a la crítica que tengamos.
- Por tanto, es útil guardar todas las configuraciones finales que simulemos.
- Dejamos pasar bastantes pasos Monte Carlo entre medidas. A temperaturas altas y bajamos podemos tomar medidas cada pocos pasos Monte Carlo. Cerca de $T_C$ hay que tomar medidas cada más tiempo, para dejar que las medidas estén menos correlacionadas. Hay que ajustar este parámetro junto con la temepratura.
    #### Eficiencia:
    - Es preferible utilizar índices de $1$ a $L^2$ $\rightarrow$ Generamos menos números aleatorios, más eficiente
    - En vez de verificar cuál es el vecino de un nodo cada vez, crearemos tablas de cuál es el vecino cada vez $\rightarrow$ Menos cuentas, más eficiencia
    - Sólo calculamos $h(j)$ una vez por cada $T$
        ##### Python:
        - Utilizar numpy siempre que sea posible
        - Evitar el uso de ciclos *for*
        - No graficar cosas, guardar los resultados y graficar a parte
    #### Características:
    - $1$ paso Monte Carlo consiste en $N$ intentos de cambio de los espines (darle en media a todos los espines una oportunidad de cambiar).
    - Condiciones de contorno periódicas $\rightarrow$ Evitamos efectos de borde (caso más cercano a $L = \inf $ )
    - Termalización: Dejamos avanzar el sistema varios pasos Monte Carlo antes de empezar a medir (no nos interesa el sistema hasta que se termaliza)
### Problemas:
- Encontraremos efectos de tamaño finito
- Además, los errores crecen en nuestra zona de interés (cerca de $T_C$). La varianza y el tiempo de correlación aumentan cerca de $T_C$, lo que aumenta el error

## Simulación

In [48]:
# importación de librerías
import numpy as np

#### Parámetros Globales

In [49]:
# Parámetros Globales
global L, N
L = 10 # Longitud de un lado del retículo cuadrado
N = L**2 # Número de nodos



In [50]:
# Función para calcular los vecinos de cada uno de los nodos de la red. SÓLO SE HACE UNA VEZ
def get_neighbors():
    n1 = [0] * (L * L)
    n2 = [0] * (L * L)
    n3 = [0] * (L * L)
    n4 = [0] * (L * L)
    
    for ix in range(1, L + 1):
        for iy in range(1, L + 1):
            i = (iy - 1) * L + ix
            ix1 = ix + 1 if ix != L else 1
            n1[i - 1] = (iy - 1) * L + ix1
            iy2 = iy + 1 if iy != L else 1
            n2[i - 1] = (iy2 - 1) * L + ix
            ix3 = ix - 1 if ix != 1 else L
            n3[i - 1] = (iy - 1) * L + ix3
            iy4 = iy - 1 if iy != 1 else L
            n4[i - 1] = (iy4 - 1) * L + ix
    
    return [n1, n2, n3, n4]
    
neighbors = get_neighbors()


#### Parámetros específicos para cada T


In [51]:

T = 2
MC = 100 # Número de pasos Monte Carlo a dar
M0 = 20 # Número de pasos Monte Carlo hasta termalizaar
